In [1]:
import pandas as pd
import os
import sys

#Path
try:
    current_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
xdrive_path = os.path.join(parent_dir, 'xdrive')
sys.path.append(xdrive_path)

pipeline_path = os.path.join(parent_dir, 'Pipeline')
sys.path.append(pipeline_path)

import get_files_from_xdrive as gxdrive

from imputation import impute_missing_values


In [2]:
data = gxdrive.read_file_from_xdrive_as_df("Prepared_Dataset_left_joint.csv")

In [3]:
# Convert DateTime and Delivery_Start_Date_Forward_Price to datetime for proper filtering
data['DateTime'] = pd.to_datetime(data['DateTime'])
data['Delivery_Start_Date_Forward_Price'] = pd.to_datetime(data['Delivery_Start_Date_Forward_Price'])

# Extract year and month for easier comparison
data['DateTime_YearMonth'] = data['DateTime'].dt.to_period('M')
data['Delivery_YearMonth'] = data['Delivery_Start_Date_Forward_Price'].dt.to_period('M')

# Create masks for current and next month
current_month_mask = data['DateTime_YearMonth'] == data['Delivery_YearMonth']
next_month_mask = data['Delivery_YearMonth'] == (data['DateTime_YearMonth'] + 1)

# Restructure the dataset to align forward prices per DateTime
# Separate current and next month rows
current_month_data = data[current_month_mask].copy()
next_month_data = data[next_month_mask].copy()

# Rename columns for clarity in the merge
current_month_data = current_month_data[['DateTime', 'Forward_Price_SE/CW(MWh)']].rename(
    columns={'Forward_Price_SE/CW(MWh)': 'Current_Month_Forward_Price'})
next_month_data = next_month_data[['DateTime', 'Forward_Price_SE/CW(MWh)']].rename(
    columns={'Forward_Price_SE/CW(MWh)': 'Next_Month_Forward_Price'}
)

# Merge current and next month data on DateTime
merged_data = pd.merge(
    current_month_data,
    next_month_data,
    on='DateTime',
    how='inner'
)

# Merge back with original features for a complete dataset
final_data = pd.merge(
    merged_data,
    data.drop(columns=['Forward_Price_SE/CW(MWh)', 'Delivery_Start_Date_Forward_Price', 
                       'DateTime_YearMonth', 'Delivery_YearMonth']),
    on='DateTime',
    how='left'
)
# Remove any remaining duplicates by keeping the first occurrence of each DateTime
final_data_unique = final_data.drop_duplicates(subset=['DateTime'])

In [4]:
# Create lagged variables for 'Average_Settlement_Price_SE/CW(MWh)'
# Lagged by 7 days and 30 days
final_data_unique['Avg_Settlement_Price_Lag7'] = final_data_unique['Average_Settlement_Price_SE/CW(MWh)'].shift(7)
final_data_unique['Avg_Settlement_Price_Lag30'] = final_data_unique['Average_Settlement_Price_SE/CW(MWh)'].shift(30)
# Apply forward and backward fill to handle missing values
final_data_unique = final_data_unique.fillna(method='ffill').fillna(method='bfill')

# Verify that there are no missing values remaining
remaining_na = final_data_unique.isnull().sum()

# Display the cleaned dataset and confirmation of no missing values
final_data_unique.info(), remaining_na[remaining_na > 0]
final_data_unique


<class 'pandas.core.frame.DataFrame'>
Index: 872 entries, 0 to 68913
Data columns (total 46 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DateTime                                     872 non-null    datetime64[ns]
 1   Current_Month_Forward_Price                  872 non-null    float64       
 2   Next_Month_Forward_Price                     872 non-null    float64       
 3   Average_Settlement_Price_SE/CW(MWh)          872 non-null    float64       
 4   Standard_Deviation_Settlement_Price_SE(MWh)  872 non-null    float64       
 5   Min_Settlement_Price_SE(MWh)                 872 non-null    float64       
 6   Max_Settlement_Price_SE(MWh)                 872 non-null    float64       
 7   Hydro_Inflow_N(MWavg)                        872 non-null    float64       
 8   Hydro_Inflow_NE(MWavg)                       872 non-null    float64       
 9   Hy

C:\Users\lilla\AppData\Local\Temp\ipykernel_31828\1092711154.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data_unique['Avg_Settlement_Price_Lag7'] = final_data_unique['Average_Settlement_Price_SE/CW(MWh)'].shift(7)
C:\Users\lilla\AppData\Local\Temp\ipykernel_31828\1092711154.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data_unique['Avg_Settlement_Price_Lag30'] = final_data_unique['Average_Settlement_Price_SE/CW(MWh)'].shift(30)
C:\Users\lilla\AppData\Local\Temp\ipykernel_31828\1092

,DateTime,Current_Month_Forward_Price,Next_Month_Forward_Price,Average_Settlement_Price_SE/CW(MWh),Standard_Deviation_Settlement_Price_SE(MWh),Min_Settlement_Price_SE(MWh),Max_Settlement_Price_SE(MWh),Hydro_Inflow_N(MWavg),Hydro_Inflow_NE(MWavg),Hydro_Inflow_S(MWavg),...,Total_capacity_N(MWavg),Total_capacity_SE/CW(MWavg),Total_capacity_S(MWavg),Consumption(MWh)_N,Consumption(MWh)_NE,Consumption(MWh)_S,Consumption(MWh)_SE/CW,Daily_Sum_Consumption(MWh),Avg_Settlement_Price_Lag7,Avg_Settlement_Price_Lag30
0,2021-05-04,219.57,325.34,236.492083,43.710734,150.42,294.99,20767.2421,2808.1711,1305.5650,...,26703.004,69157.78010,20792.616,142891.2,265531.2,280231.2,942040.8,1630694.4,236.492083,236.492083
45,2021-05-05,213.72,297.29,235.216667,43.157630,148.38,287.27,20595.1560,2790.8320,1407.9316,...,26703.004,69157.78010,20792.616,142891.2,265531.2,280231.2,942040.8,1630694.4,236.492083,236.492083
90,2021-05-06,210.29,300.57,237.091667,41.620428,149.17,296.04,20456.3201,2781.1281,1614.7059,...,26703.004,69177.78010,20792.616,142891.2,265531.2,280231.2,942040.8,1630694.4,236.492083,236.492083
135,2021-05-07,212.41,310.57,234.381667,37.611976,146.16,272.77,20368.4398,2757.8587,1923.3784,...,26703.004,69177.78010,20792.616,142891.2,265531.2,280231.2,942040.8,1630694.4,236.492083,236.492083
180,2021-05-10,215.50,299.19,217.603750,39.811635,151.95,263.55,20187.3170,2679.6654,1852.6125,...,26703.004,69177.78010,20792.616,140860.8,261746.4,280740.0,930818.4,1614165.6,236.492083,236.492083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67889,2024-10-03,543.00,412.23,716.798333,225.274395,569.05,1143.33,984.1782,1065.8911,13855.8395,...,27249.024,77849.56830,21241.884,205785.6,324885.6,317407.2,1152991.2,2001069.6,314.664583,250.937917
68145,2024-10-04,510.79,341.62,607.692500,19.158619,586.39,643.15,957.8848,1046.8771,13023.3871,...,27249.024,77849.56830,21241.884,205785.6,324885.6,317407.2,1152991.2,2001069.6,321.145417,253.929167
68401,2024-10-05,510.79,341.62,437.552917,154.565069,130.86,562.54,937.1469,1028.8140,10701.9223,...,27249.024,77890.16834,21241.884,205005.6,329930.4,323688.0,1102269.6,1960893.6,612.153333,285.280833
68657,2024-10-06,510.79,341.62,358.549167,229.036164,61.07,572.47,923.9403,1015.1727,8564.7942,...,27249.024,77890.16834,21241.884,205005.6,329930.4,323688.0,1102269.6,1960893.6,421.240000,250.927083


In [5]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

In [6]:
split_point = '2023-09-09'
train_data = final_data_unique[final_data_unique['DateTime'] < split_point].copy()
valid_data = final_data_unique[final_data_unique['DateTime'] >= split_point].copy()

In [7]:
target_column = "Current_Month_Forward_Price"
# Sort data by DateTime to preserve time series order
if 'DateTime' in train_data.columns:
        train_data['DateTime'] = pd.to_datetime(train_data['DateTime'])
        train_data = train_data.sort_values('DateTime')
        train_data = train_data.set_index('DateTime')

# Split into features (X) and target (y)
X_train = train_data.drop(columns=[target_column]) 
y_train = train_data[target_column]  

In [8]:
X_train

,Next_Month_Forward_Price,Average_Settlement_Price_SE/CW(MWh),Standard_Deviation_Settlement_Price_SE(MWh),Min_Settlement_Price_SE(MWh),Max_Settlement_Price_SE(MWh),Hydro_Inflow_N(MWavg),Hydro_Inflow_NE(MWavg),Hydro_Inflow_S(MWavg),Hydro_Inflow_SE/CW(MWavg),Daily_Sum_Hydro_Inflow(MWavg),...,Total_capacity_N(MWavg),Total_capacity_SE/CW(MWavg),Total_capacity_S(MWavg),Consumption(MWh)_N,Consumption(MWh)_NE,Consumption(MWh)_S,Consumption(MWh)_SE/CW,Daily_Sum_Consumption(MWh),Avg_Settlement_Price_Lag7,Avg_Settlement_Price_Lag30
DateTime,,,,,,,,,,,,,,,,,,,,,
2021-05-04,325.34,236.492083,43.710734,150.42,294.99,20767.2421,2808.1711,1305.5650,29298.6262,54179.6044,...,26703.004,69157.78010,20792.616,142891.2,265531.2,280231.2,942040.8,1630694.4,236.492083,236.492083
2021-05-05,297.29,235.216667,43.157630,148.38,287.27,20595.1560,2790.8320,1407.9316,28676.2038,53470.1234,...,26703.004,69157.78010,20792.616,142891.2,265531.2,280231.2,942040.8,1630694.4,236.492083,236.492083
2021-05-06,300.57,237.091667,41.620428,149.17,296.04,20456.3201,2781.1281,1614.7059,28024.9451,52877.0992,...,26703.004,69177.78010,20792.616,142891.2,265531.2,280231.2,942040.8,1630694.4,236.492083,236.492083
2021-05-07,310.57,234.381667,37.611976,146.16,272.77,20368.4398,2757.8587,1923.3784,27438.5920,52488.2689,...,26703.004,69177.78010,20792.616,142891.2,265531.2,280231.2,942040.8,1630694.4,236.492083,236.492083
2021-05-10,299.19,217.603750,39.811635,151.95,263.55,20187.3170,2679.6654,1852.6125,26166.8111,50886.4060,...,26703.004,69177.78010,20792.616,140860.8,261746.4,280740.0,930818.4,1614165.6,236.492083,236.492083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,69.70,69.040000,0.000000,69.04,69.04,1879.5282,2028.8337,4882.3822,18838.6581,27629.4022,...,27240.888,75468.41102,21109.634,183252.0,288266.4,287217.6,961884.0,1720620.0,69.040000,69.040000
2023-09-04,69.72,69.040000,0.000000,69.04,69.04,1950.5496,2035.4973,33749.7321,19009.5726,56745.3516,...,27240.888,75468.41102,21109.634,180168.0,289876.8,285326.4,969585.6,1724956.8,69.040000,69.040000
2023-09-05,69.64,69.040000,0.000000,69.04,69.04,1945.5669,2054.0844,43234.1412,19411.5381,66645.3306,...,27240.888,75468.41102,21109.634,180168.0,289876.8,285326.4,969585.6,1724956.8,69.040000,69.040000


In [9]:
from sklearn.discriminant_analysis import StandardScaler


scaler = StandardScaler()
#Scale training data and transform all datasets
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)

In [10]:
X_train_scaled

,Next_Month_Forward_Price,Average_Settlement_Price_SE/CW(MWh),Standard_Deviation_Settlement_Price_SE(MWh),Min_Settlement_Price_SE(MWh),Max_Settlement_Price_SE(MWh),Hydro_Inflow_N(MWavg),Hydro_Inflow_NE(MWavg),Hydro_Inflow_S(MWavg),Hydro_Inflow_SE/CW(MWavg),Daily_Sum_Hydro_Inflow(MWavg),...,Total_capacity_N(MWavg),Total_capacity_SE/CW(MWavg),Total_capacity_S(MWavg),Consumption(MWh)_N,Consumption(MWh)_NE,Consumption(MWh)_S,Consumption(MWh)_SE/CW,Daily_Sum_Consumption(MWh),Avg_Settlement_Price_Lag7,Avg_Settlement_Price_Lag30
DateTime,,,,,,,,,,,,,,,,,,,,,
2021-05-04,1.365083,0.821147,5.711877,0.259575,1.148478,0.652677,-0.595618,-0.913124,-0.405810,-0.381725,...,-0.751626,-1.783591,-1.683335,-0.967846,-0.931188,-0.517277,-0.498709,-0.801970,0.803323,0.746835
2021-05-05,1.169805,0.812283,5.635319,0.244852,1.096975,0.637349,-0.598513,-0.902204,-0.434507,-0.401637,...,-0.751626,-1.783591,-1.683335,-0.967846,-0.931188,-0.517277,-0.498709,-0.801970,0.803323,0.746835
2021-05-06,1.192640,0.825315,5.422548,0.250554,1.155483,0.624983,-0.600134,-0.880146,-0.464534,-0.418281,...,-0.751626,-1.772437,-1.683335,-0.967846,-0.931188,-0.517277,-0.498709,-0.801970,0.803323,0.746835
2021-05-07,1.262258,0.806480,4.867719,0.228831,1.000239,0.617155,-0.604019,-0.847219,-0.491568,-0.429194,...,-0.751626,-1.772437,-1.683335,-0.967846,-0.931188,-0.517277,-0.498709,-0.801970,0.803323,0.746835
2021-05-10,1.183033,0.689871,5.172184,0.270617,0.938729,0.601022,-0.617076,-0.854768,-0.550204,-0.474151,...,-0.751626,-1.772437,-1.683335,-1.137749,-1.269966,-0.495034,-0.774572,-1.047039,0.803323,0.746835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,-0.414625,-0.342664,-0.338328,-0.327747,-0.358922,-1.029688,-0.725755,-0.531567,-0.888073,-1.126881,...,1.278969,1.735950,0.992367,2.409531,1.103839,-0.211846,-0.010933,0.531335,-0.356455,-0.401758
2023-09-04,-0.414486,-0.342664,-0.338328,-0.327747,-0.358922,-1.023362,-0.724643,2.547853,-0.880193,-0.309715,...,1.278969,1.735950,0.992367,2.151463,1.247985,-0.294525,0.178383,0.595636,-0.356455,-0.401758
2023-09-05,-0.415043,-0.342664,-0.338328,-0.327747,-0.358922,-1.023805,-0.721539,3.559601,-0.861660,-0.031863,...,1.278969,1.735950,0.992367,2.151463,1.247985,-0.294525,0.178383,0.595636,-0.356455,-0.401758


In [11]:
if 'DateTime' in valid_data.columns:
        valid_data['DateTime'] = pd.to_datetime(valid_data['DateTime'])
        valid_data = valid_data.sort_values('DateTime')
        valid_data = valid_data.set_index('DateTime')
# Split into features (X) and target (y)
X_val = valid_data.drop(columns=[target_column]) 
y_val = valid_data[target_column]  

X_val_scaled = pd.DataFrame(scaler.fit_transform(X_val), columns=X_val.columns, index=X_val.index)

In [12]:
X_val_scaled

,Next_Month_Forward_Price,Average_Settlement_Price_SE/CW(MWh),Standard_Deviation_Settlement_Price_SE(MWh),Min_Settlement_Price_SE(MWh),Max_Settlement_Price_SE(MWh),Hydro_Inflow_N(MWavg),Hydro_Inflow_NE(MWavg),Hydro_Inflow_S(MWavg),Hydro_Inflow_SE/CW(MWavg),Daily_Sum_Hydro_Inflow(MWavg),...,Total_capacity_N(MWavg),Total_capacity_SE/CW(MWavg),Total_capacity_S(MWavg),Consumption(MWh)_N,Consumption(MWh)_NE,Consumption(MWh)_S,Consumption(MWh)_SE/CW,Daily_Sum_Consumption(MWh),Avg_Settlement_Price_Lag7,Avg_Settlement_Price_Lag30
DateTime,,,,,,,,,,,,,,,,,,,,,
2023-09-11,-0.516613,-0.364684,-0.377023,-0.282141,-0.410082,-0.760902,-0.488322,-0.036897,-0.452874,-0.553170,...,-1.082152,-1.674053,-0.939738,-0.080799,-1.372497,-1.262569,-0.767540,-1.076373,-0.355480,-0.262370
2023-09-12,-0.516613,-0.364684,-0.377023,-0.282141,-0.410082,-0.765887,-0.478881,-0.237974,-0.481625,-0.677680,...,-1.082152,-1.681378,-0.939738,-0.080799,-1.372497,-1.262569,-0.767540,-1.076373,-0.355480,-0.262370
2023-09-13,-0.516613,-0.364684,-0.377023,-0.282141,-0.410082,-0.769369,-0.467545,-0.202161,-0.511587,-0.673706,...,-1.082152,-1.681378,-0.939738,-0.080799,-1.372497,-1.262569,-0.767540,-1.076373,-0.355480,-0.262370
2023-09-14,-0.515573,-0.364684,-0.377023,-0.282141,-0.410082,-0.771761,-0.462855,-0.186518,-0.535603,-0.678040,...,-1.082152,-1.681378,-0.939738,-0.080799,-1.372497,-1.262569,-0.767540,-1.076373,-0.355480,-0.262370
2023-09-15,-0.516266,-0.364684,-0.377023,-0.282141,-0.410082,-0.776337,-0.473405,-0.283803,-0.559370,-0.746063,...,-1.082152,-1.681378,-0.939738,-0.080799,-1.372497,-1.262569,-0.767540,-1.076373,-0.355480,-0.262370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-03,2.454602,5.822704,4.729173,5.476320,5.187559,-0.864505,-0.813817,-0.191145,-1.273921,-1.143165,...,0.924085,1.814280,1.823182,2.570419,0.990421,-0.200189,1.472165,1.337343,2.812857,4.689603
2024-10-04,1.842555,4.780525,0.057237,5.676020,2.581347,-0.867915,-0.819746,-0.251900,-1.284038,-1.183236,...,0.924085,1.814280,1.823182,2.570419,0.990421,-0.200189,1.472165,1.337343,2.896454,4.771037
2024-10-05,1.842555,3.155352,3.126436,0.429821,2.161324,-0.870605,-0.825378,-0.421328,-1.291873,-1.280758,...,0.924085,1.873758,1.823182,2.471277,1.426186,0.078868,0.554325,0.842316,6.650195,5.624552


In [13]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
import numpy as np

# Create XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100)
model.fit(X_train_scaled, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [16]:
# Predictions and evaluation
y_pred = model.predict(X_val_scaled)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')


Mean Squared Error: 1769.8198853343813
Root Mean Squared Error: 42.06922729661648


In [17]:

from sklearn.metrics import r2_score
# R-squared score
r2 = r2_score(y_val, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R-squared: {r2}')
model_performance.append({
    'Model': model,
    'Mean Squared Error': mse,
    'Root Mean Squared Error': rmse,
    'R-squared': r2})

Mean Squared Error: 1769.8198853343813
Root Mean Squared Error: 42.06922729661648
R-squared: 0.7771480844266478


NameError: name 'model_performance' is not defined